# Imports

In [ ]:
import sys

In [ ]:
sys.path.append("../")

In [ ]:
import config
import plant.manipulator as manipulator
import constants
import plant.simulation as simulation
from ctrl.impedance_generators.setpoint_generators.circle import CircularSetpointGenerator

In [ ]:
import pydrake
from pydrake.all import (
    MultibodyPlant, RotationMatrix, RigidTransform, RollPitchYaw, Meshcat, MeshcatVisualizerParams, DiagramBuilder,
    MeshcatVisualizerCpp
)
from pydrake.common.eigen_geometry import Quaternion_

In [ ]:
import numpy as np
from numpy import array

In [ ]:
from collections import namedtuple

In [ ]:
from visualization import AddMeshcatTriad

In [ ]:
import panda.panda_config

# Setup

In [ ]:
meshcat = Meshcat()
web_url = meshcat.web_url()

builder = pydrake.systems.framework.DiagramBuilder()

# Physical parameters
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=config.DT)
manipulator.addArm(
    plant,
    scene_graph=scene_graph,
    m_M=constants.nominal_sys_consts.m_M,
    r=constants.nominal_sys_consts.r,
    mu=constants.nominal_sys_consts.mu
)
plant.Finalize()

meshcat_params = MeshcatVisualizerParams()
vis = MeshcatVisualizerCpp.AddToBuilder(
    builder, scene_graph.get_query_output_port(), meshcat, meshcat_params)

diagram = builder.Build()

In [ ]:
diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_context)
vis_context = vis.GetMyContextFromRoot(diagram_context)

In [ ]:
def convert_config(config_dict):
    pref = "panda_joint"
    out = []
    for i in range(7):
        out.append(config_dict[pref + str(i+1)])
    return out

In [ ]:
def convert_pose_dict(pos_dict):
    pos = pos_dict['position']
    quat_ = pos_dict['orientation']
    quat = Quaternion_[float](w=quat_.w, x=quat_.x, y=quat_.y, z=quat_.z)
    return pos, quat

# Evaluate single data point positions

## Set $q_0$

In [ ]:
q0 = manipulator.neutral_q

In [ ]:
plant.SetPositions(plant_context, q0)

In [ ]:
quaternion = namedtuple("quaternion", "w x y z")

## Evaluate panda hand position

In [ ]:
X = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))
X

In [ ]:
quat = X.rotation().ToQuaternion()
print("x: {}\ny: {}\nz: {}\nw: {}".format(
    quat.x(),
    quat.y(),
    quat.z(),
    quat.w()
))

## Evaluate end effector position

In [ ]:
X = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X

In [ ]:
quat = X.rotation().ToQuaternion()
print("x: {}\ny: {}\nz: {}\nw: {}".format(
    quat.x(),
    quat.y(),
    quat.z(),
    quat.w()
))

# Calculate offset transform

## Raw inputs

In [ ]:
q_1_dict = {'panda_joint1': 0.5920717059436604, 'panda_joint2': -1.1923560818855956, 
            'panda_joint3': -0.9708204979467376, 'panda_joint4': -2.7506264015373474, 
            'panda_joint5': -0.029033820947907442, 'panda_joint6': 2.930613038685587, 
            'panda_joint7': -0.02070499410846963}
q_1 = convert_config(q_1_dict)

endpoint_dict_1 = {
    'position': array([ 0.42532115, -0.14015937,  0.48573156]),
    'orientation': quaternion(-0.154862189461281, 0.786106686594126, 0.138303084147821, 0.582173716756831)}
pos_1, quat_1 = convert_pose_dict(endpoint_dict_1)

In [ ]:
q_2_dict = {
    'panda_joint1': -0.34472184209238016, 'panda_joint2': 0.9388866558695357,
    'panda_joint3': 0.9880253082911173, 'panda_joint4': -0.7271104281800026,
    'panda_joint5': -0.0012154511641710997, 'panda_joint6': 3.7257856980155495,
    'panda_joint7': 0.8182791094515057}
q_2 = convert_config(q_2_dict)

endpoint_dict_2 = {
    'position': array([ 0.79588991, -0.09137514,  0.7548383 ]),
    'orientation': quaternion(-0.314319532035134, 0.410839113601719, -0.265100815312353, 0.813717403179319)}
pos_2, quat_2 = convert_pose_dict(endpoint_dict_2)

In [ ]:
q_3_dict = {'panda_joint1': -0.19806813592241537, 'panda_joint2': -1.4399713259579843,
            'panda_joint3': 1.1155892525723103, 'panda_joint4': -3.06285333814119,
            'panda_joint5': 0.2876149759428815, 'panda_joint6': 1.8047770896460589,
            'panda_joint7': 0.9565096986823611}
q_3 = convert_config(q_3_dict)

endpoint_dict_3 = {
    'position': array([0.2077336 , 0.00865574, 0.27209928]), 
    'orientation': quaternion(0.391679109975748, 0.914522326921643, -0.0711655063304001, 0.0719156386272558)}
pos_3, quat_3 = convert_pose_dict(endpoint_dict_3)

In [ ]:
q_4_dict = {'panda_joint1': 0.13736649785340663, 'panda_joint2': -1.511650379851549, 
            'panda_joint3': -0.13755869549826572, 'panda_joint4': -1.5583121646141103, 
            'panda_joint5': 0.2808405818541844, 'panda_joint6': 0.6858634739555333, 
            'panda_joint7': 0.8222335470252566}
q_4 = convert_config(q_4_dict)

endpoint_dict_4 = {
    'position': array([-0.1855927 , -0.01979181,  0.70970675]),
    'orientation': quaternion(-0.140598016690583, 0.928419729813831, 0.152047280259162, 0.308464953537479)}
pos_4, quat_4 = convert_pose_dict(endpoint_dict_4)

In [ ]:
neutral_quat = Quaternion_[float](
    w=8.86055883171457e-05,
    x=0.999999994645199,
    y=-4.8259523485426e-05,
    z=-2.30145461781558e-05
)
neutral_q = manipulator.neutral_q
neutral_pos = [3.06918839e-01, 2.42611677e-05, 4.86808120e-01]

## Sanity check visuals

In [ ]:
plant.SetPositions(plant_context, neutral_q)

In [ ]:
vis.Publish(vis_context)

In [ ]:
X_P_SP = RigidTransform(quaternion=neutral_quat, p=neutral_pos)
X_W_SP = manipulator.X_W_panda.multiply(X_P_SP)

In [ ]:
AddMeshcatTriad(meshcat, "end_effector_pose", X_PT=X_W_SP)

## Calculate offset

In [ ]:
qs = [neutral_q, q_1, q_2, q_3, q_4]

quaternions = [
    neutral_quat, quat_1, quat_2, quat_3, quat_4
]

positions = [
    neutral_pos, pos_1, pos_2, pos_3, pos_4
]

assert(len(positions) == len(quaternions))
assert(len(positions) == len(qs))

In [ ]:
X_pred_trues = []
for (q, quat, pos) in zip(qs, quaternions, positions):
    X_panda_true = RigidTransform(
        p=pos, quaternion=quat
    )
    X_W_true = manipulator.X_W_panda.multiply(X_panda_true)
    
    plant.SetPositions(plant_context, q)
    X_W_pred = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
    
    X_pred_true = X_W_pred.inverse().multiply(X_W_true)
    X_pred_trues.append(X_pred_true)

In [ ]:
for X_pred_true in X_pred_trues:
    rpy = RollPitchYaw(X_pred_true.rotation()).vector()
    xyz = X_pred_true.translation()
    print("x: {:.5f}\ty: {:.5f}\tz: {:.5f}\t\tr: {:.5f}\tp: {:.5f}\ty: {:.5f}".format(
        xyz[0], xyz[1], xyz[2], rpy[0], rpy[1], rpy[2]
    ))

# Double check conversion

## Get setpoint corresponding to neutral in simulation

In [ ]:
plant.SetPositions(plant_context, manipulator.neutral_q)
# Pose from world to setpoint in software
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))

In [ ]:
# Pose from panda to setpoint in hardware
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW

## Get setpoint corresponding to neutral in hardware

Result of `arm.endpoint_pose()`:

In [ ]:
X_panda_SP_HW__real = None

## Compare

# Test a few candidate points

## Starting position

In [ ]:
starting_q = q = [
    -1.90831538,
    -0.74013834,
    2.29209901,
    -2.58333082,
    4.52633766,
    0.60032527,
    -3.10678351
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.1`

In [ ]:
t1_q = q = [
    -2.1146478801194806,
    -0.5741377797591646,
    2.421334309172595,
    -2.6520921048533364,
    4.296990530955979,
    0.4557018857881668,
    -3.2641926011421214,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.2`

In [ ]:
t2_q = q = [
    -1.9923999435056001,
    -0.3649667563328254,
    2.2852638372758345,
    -2.834742287573601,
    3.93642474172192,
    0.5108014249814706,
    -3.709962619794525,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.3`

In [ ]:
t3_q = q = [
    -1.3669552018880975,
    -0.0493623881307237,
    1.508231366013591,
    -2.985729614242096,
    3.3298860836292303,
    0.6243204293551693,
    -4.314615141139232,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.4`

In [ ]:
t4_q = q = [
    -0.8343593302307178,
    -0.5113238160486787,
    1.15297441506536,
    -3.036362269755852,
    3.729384595352719,
    1.105484178182587,
    -4.152855198300328,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.5`

In [ ]:
t5_q = q = [
    -0.6739151219982793,
    -0.8307647774224596,
    1.0289509907170764,
    -3.056506301627166,
    3.7113407918053385,
    1.5572295627007329,
    -4.24757843211469,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.6`

In [ ]:
t6_q = q = [
    -0.6374298041563494,
    -1.051983578537568,
    0.9552403158630598,
    -3.0243824117891935,
    3.593330117640885,
    1.9333055716922152,
    -4.325931578828055,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.7`

In [ ]:
t7_q = q = [
    -0.6985304670801346,
    -1.2241447167059634,
    0.9516565461351049,
    -2.945916103740626,
    3.421652194372958,
    2.2480143731924884,
    -4.298986656717967,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.8`

In [ ]:
t8_q = q = [
    -0.791058996058301,
    -1.3340007077373213,
    0.9761776658876062,
    -2.8500472338816167,
    3.166709249486946,
    2.459191495106102,
    -4.167089374724142,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time * 0.9`

In [ ]:
t9_q = q = [
    -0.8688123684145203,
    -1.3838866414859068,
    0.9972242249030413,
    -2.7682700094431887,
    2.842237591892566,
    2.558597770370147,
    -3.987070127648293,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

## `t = end_time`

In [ ]:
t_end_q = q = [
    -0.9404502731817744,
    -1.4239662327751528,
    1.0110803170708464,
    -2.717567209177025,
    2.510573875661588,
    2.5518798052855085,
    -3.7958517758859918,
]

In [ ]:
plant.SetPositions(plant_context, q)
X_W_SP_SW = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("main_end_effector_body"))
X_panda_SP_HW = panda.panda_config.X_panda_SP_hw(X_W_SP_SW)

In [ ]:
X_panda_SP_HW.translation()

In [ ]:
quat_ = X_panda_SP_HW.rotation().ToQuaternion()
print("w: {}\nx: {}\ny: {}\nz: {}".format(quat_.w(), quat_.x(), quat_.y(), quat_.z()))

# Generate trajectory

## Sanity check speed

In [ ]:
end_time = 180
desired_radius=constants.nominal_sys_consts.w_L/2

In [ ]:
setpoint_gen = CircularSetpointGenerator(
    desired_radius=desired_radius, 
    end_time=end_time,
    sys_consts=constants.nominal_sys_consts
)

In [ ]:
angular_vel = np.pi/end_time
tangential_vel = angular_vel*desired_radius
tangential_vel

In [ ]:
times = np.arange(0, end_time, 0.1)
x0s = np.zeros((len(times), 6))

In [ ]:
for i, t in enumerate(times):
    x0s[i,:] = setpoint_gen._calc_x0(t)

In [ ]:
positions = x0s[:,3:]
rpys = x0s[:,:3]

In [ ]:
quaternions = np.zeros((len(times), 4))
for i, t in enumerate(times):
    quat = RollPitchYaw(rpys[0,:]).ToQuaternion()
    quaternions[i,0] = quat.w()
    quaternions[i,1] = quat.x()
    quaternions[i,2] = quat.y()
    quaternions[i,3] = quat.z()

In [ ]:
q_ = quat_(quaternions[0,0], quaternions[0,1], quaternions[0,2], quaternions[0,3])